# ESM3 Inverse Folding Notebook

This notebook is intended to be used as a tool for inverse folding using the ESM3 model.


### Setup

Install dependencies and setup the colab environment for asyncio requests


In [ ]:
!pip install git+https://github.com/evolutionaryscale/esm
!pip install pydssp pygtrie dna-features-viewer nest_asyncio py3dmol

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Inference Settings


In [ ]:
from esm.widgets.utils.types import ClientInitContainer
from esm.widgets.views.inverse_folding import (
    create_inverse_folding_ui,
)
from esm.widgets.views.login import create_login_ui

In [ ]:
client_init = ClientInitContainer()
create_login_ui(client_init)

We need to log into huggingface if using the model locally


In [ ]:
from esm.utils.misc import huggingfacehub_login

if client_init.metadata["inference_option"] == "Local":
    huggingfacehub_login()

## Inverse Folding UI

If running on Google colab, it is recommended to use the light theme and select the "View output fullscreen" option in the cell toolbar for the best experience.


In [ ]:
client = client_init()
create_inverse_folding_ui(client)